__Students:__ Andrea Ceriati, Sarah Demmon, Kat McEldowney, Chantal Thomas 

__Instructor:__ Daniel Pulido-Mendez

__Institution:__ Columbia Engineering

__Course:__ Data Analytics Boot Camp

__Assignment:__ Python Project

__Date:__ 12/18/2023

# Python Project

## Table of Contents


   [**Introduction**](#Intro1)

   [**Importing Libraries**](#IL)

1. [**Data Import, Cleaning & Wrangling**](#DWrang)

    1.1 [**Importing Census Data**](#CensusD)
    
    1.2 [**Importing Tax Data**](#TaxD)

    1.3 [**Merge the Data**](#DataD)

    1.4 [**Handling Missing Data**](#MissD)

    1.5 [**Checking and Correcting Data Format**](#FormD)

2. [**Exploratory Data Analysis**](#EDA)

    2.1 [**Descriptive Statistics**](#DS)

    2.2 [**Correlation**](#Corr)

    2.3 [**Data Visualization**](#DV)

3. [**Conclusions**](#Concl)

## Introduction <a name="Intro1"></a>

New York imposes a mortgage tax on real estate transactions. This tax rate in NYC is 1.8% on mortgage amounts less than $500,000 and 1.95% on mortgages over $500,000. When buying a home, there is no sales tax, you are only taxed on the mortgage amount. Thre is a mansion Tax of 1% on homes over $1,000,000.

This means that a person buying a $500,000 home for cash pays no tax, a person buying the same home and putting 20% down, (or borrowing $400,000) pays $7,200 in mortgage tax, and a person who only has a 3% down payment on the same purchase must pay $8,730 in additional closing costs to cover this tax.

Upon examining the NY.gov page regarding sales tax collection (https://www.tax.ny.gov/data/stats/taxfacts/sales-miscellaneous-taxes.htm) it was discovered that "New York State offers tax exemptions on food, clothing, residential energy, Internet charges, and cable television, as well as a motor fuel tax cap.' It could be inferred that the reason for the lack of a sales tax on Real Estate puchases is becuase the government sees a home purchase as a necessary purchase that should be untaxed. If this is the case, why are mortgges taxed? Any why aren't purchases of investment properties taxed?
Real Estate is the only industry that we can think of where you can purchase corporate assets and not pay tax on that purchase. A real estate investor who purchases buildings for cash, does not pay any sales tax on this transaction. The US government is attempting to limit corporate control of single family homes with the pending legislation *End Hedge Fund Control of American Homes Act of 2023* that will require any entity with a net value including help assets of over $50 billion to sell off the single family homes they currently own over the next ten years, and prohibit them from aquiring new single family homes. This is an effort to increase the avaialbily of single family homes for individual buyers. There is anohter piece of legislation proposed that would charge a $10,000 per home annual fee for investors owning over 75 homes. (https://www.nytimes.com/2023/12/06/realestate/wall-street-housing-market.html) (https://www.businessinsider.com/housing-market-affordability-investors-hedge-funds-wall-street-democrats-bill-2023-12). In light of this, the ability to purchase a single family home as an invetment without paying sales tax seems like a big oversight. We propose, to assist in this goal of promoting individual ownership of single family homes and apartments, a sales tax of 50% of the current sales tax in NYC, be levied on purchases of single family homes by investors.

We have two hypotheses that we will explore in this project: 
•	We believe that the mortgage Tax is a policy that has a disparate impact on people who are already economically disadvantaged. 
•	We also believe that if this tax was lowered and applied across the board to all home purchases, with a higher tax rate applied to investment propery purchases, based on sales price not mortgage amount, the policy would not only be more equitable, but also raise more revenue for the government.

We analyzed the sales data of 3012 homes sold in Brooklyn, NY in 2022. We used this data to calculate how much mortgage tax was raised and how much would be raised if it were instead a sales tax that was only 1/2 of the current mortgage tax rate and 1/2 the current mansion tax rate on purchases of a proimary residence, and 1/2 the current sales tax rate on investment property purchases. 
We also used census data to examine the differences in demographics of the different zip codes to see if people with lower incomes are paying a larger portion of this tax than people living in zip codes with higher average incomes.

The property sales data came from Property Radar (https://app.propertyradar.com/). We filtered the search to only included sales of condos and single family homes that closed in the 2022 calendar year and market sales that were listed on the open market. This means that the sample does not include short sales, forclosures, or arms length transactions (such as a preson selling to a family member or friend without first listing the home on the open market).

In [ ]:
import pandas as pd
import numpy as np
import requests
from census import Census
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import json
import plotly.express as px
from urllib.request import urlopen

import warnings
warnings.filterwarnings("ignore")

## 1. Data Cleaning & Wrangling <a name="DWrang"></a>

### 1.1 Importing Census Data <a name="CensusD"></a>

The dataset was obtained from the United States Census Bureau. To access the data, an API call was initiated. The dataset provides information on population, median age, household income, median gross rent, categorized by zipcode, for the year 2022.
The selected zip codes encompassed all of Brooklyn.

Information about the data and its variables can be found at links below:

* Review the following page to learn more about the Census API: <https://www.census.gov/programs-surveys/acs/technical-documentation/table-and-geography-changes/2022/5-year.html>

* Review the following page to learn more about the 2022 data labels: <https://api.census.gov/data/2022/acs/acs5/variables.html>

In [ ]:
# Import U.S. Census API Key
from config import api_key

# Create an instance of the Census library
c = Census(
    api_key,
    year = 2022
)

# Run Census Search to retrieve data on all Brooklyn Zipcodes (2022 ACS5 Census)
census_data = c.acs5.get(
    (
        "NAME",
        "B19013_001E",
        "B01003_001E",
        "B01002_001E",
        "B25064_001E",
        "B25089_001E"
    ),
    {'for': 'zip code tabulation area:*'}
)

# Convert to DataFrame
census_df = pd.DataFrame(census_data)

# Column renaming
census_df = census_df.rename(
    columns = {
        "B01003_001E": "Population",
        "B01002_001E": "Median Age",
        "B19013_001E": "Household Income",
        "B25064_001E": "Median Gross Rent",
        "NAME": "Name",
        "zip code tabulation area": "Zipcode"
    }
)

# Configure the final DataFrame
census_df = census_df[
    [
        "Zipcode",
        "Population",
        "Median Age",
        "Household Income",
        "Median Gross Rent",
    ]
]

# Filter for just the considered zipcodes
zipcodes = ['11201', '11202', '11203', '11204','11205','11206','11207','11208',
'11209','11210','11211','11212','11213','11214','11215','11216','11217','11218',
'11219','11220','11221','11222','11223','11224','11225','11226','11228','11229',
'11230','11231','11232','11233','11234','11235','11236','11237','11238','11239',
'11240','11241','11242','11243','11247','11249','11252','11252','11256']

census_df=census_df.loc[census_df['Zipcode'].isin(zipcodes)]
census_df.reset_index(drop=True, inplace=True)

### 1.2 Importing Tax Data <a name="TaxD"></a>

The property sales data was downloaded from Property Radar. The data was filtered to only included sales of condos and single family homes that closed in the 2022 calendar year and market sales that were listed on the open market. This means that the sample does not include short sales, forclosures, or arms length transactions (such as a preson selling to a family member or friend without first listing the home on the open market).
The dataset includes salses data of 3012 homes sold in Brooklyn, NY in 2022.

In [ ]:
# Import the data and create one dataset
full_df = pd.read_csv('Data/brooklyn_2022_sales.csv')

In [ ]:
# Fill NaN values with 0
full_df = full_df.fillna(0)

# Rename 1st Amount Column. Add a count column
full_df.rename(columns={'1st Amount': 'Amount Mortgaged'}, inplace=True)
full_df['Count'] = 1 # Counter
full_df = full_df.loc[:, ['Count', 'ZIP', 'Owner Occ?', 'Purchase Down %', 
                                 'Purchase Amt', 'Amount Mortgaged']]

In [ ]:
# Calculate how much mortgage tax was collected.
# For mortgages less than $500,000, tax is 1.8% for mortgages of $500,000, 1.95% 
# Calculate the mansion tax as 1% for sales over $1,000,000
full_df['Mtg Tx Pd'] = np.where(full_df['Amount Mortgaged'] >= 500000, 
                                 full_df['Amount Mortgaged'] * 0.0195, full_df['Amount Mortgaged'] * 0.018)
full_df['Mansion Tax Pd'] = np.where(full_df['Purchase Amt'] >= 1000000, full_df['Purchase Amt'] * 0.01, 0)

In [ ]:
# Created two dataframes, one for investor purchases and the other for primary residences
investor_df = full_df[(full_df['Owner Occ?'] ==1)]
owner_occ_df = full_df[(full_df['Owner Occ?'] == 0)]

In [ ]:
# Calculate new sales tax info for private residences, 0.9% for purchases under $1mil and 
# Calculate new sales tax for investment purchases, 4.4357%
owner_occ_df['New Sales Tax Amount'] = np.where(owner_occ_df['Purchase Amt'] >= 1000000, owner_occ_df['Purchase Amt'] * 0.009,
                                owner_occ_df['Purchase Amt'] * 0.014)
investor_df['New Sales Tax Amount'] = investor_df['Purchase Amt'] * 0.04438

In [ ]:
# Concatenated the two files
frames = [owner_occ_df, investor_df]
calculated_full_df = pd.concat(frames)
print(f"Number of rows and columns in the DataFrame: {calculated_full_df.shape}")

### 1.3 Merge the Data <a name="DataD"></a>

The two datasets were merged to form a unified dataframe for the final analysis.

In [ ]:
# Rename ZIP column in calculated_full_df
calculated_full_df = calculated_full_df.rename(
    columns = {
        "ZIP": "Zipcode",
    }
)

# change datatype
census_df['Zipcode']=census_df['Zipcode'].astype('float64')

In [ ]:
# Combine the data into a single DataFrame
complete_df = pd.merge(calculated_full_df, census_df, how="left", on=("Zipcode"))

In [ ]:
# prints the size of the dataframe
print('The size of the data frame is:',complete_df.shape)

### 1.4 Handling Missing Data <a name="MissD"></a>

The dataset was carefully examined to identify any missing data. During the analysis, it was noted that there were entries displaying a zipcode of 0, which is not valid for Brooklyn. Additionally, there was a zipcode, 11243, representing just one block in Brooklyn, for which no census data was available. This discovery is intriguing and necessitates further investigation.

In [ ]:
# Verify if the data contains missing values
missing_data=complete_df.isnull()
for column in missing_data.columns:
    print(column)
    print(missing_data[column].value_counts()) 
    print("")

In [ ]:
# Print rows with missing values
complete_df[complete_df.isnull().any(axis=1)]

In [ ]:
# Drop rows with Nan values
complete_df.drop(index=[583,584,585,586,587,588,589,590,591,
                        1921,1922,1923,1924,1925,1926], inplace=True)

In [ ]:
# print number of missing data in the dataset
print('The number of missing data in the dataset is:',complete_df.isnull().sum().sum())

In [ ]:
# prints the size of the dataframe
print('The size of the data frame is:',complete_df.shape)

### 1.5 Checking and Correcting Data Format <a name="FormD"></a>

The data format was checked and corrected to ensure data consistency.

In [ ]:
# Check datatypes
complete_df.dtypes

In [ ]:
# Change datatype for zipcode
complete_df['Zipcode']=complete_df['Zipcode'].astype('object')

In [ ]:
# Reset indeces
complete_df.reset_index(drop=True, inplace=True)

In [ ]:
complete_df.head()

In [ ]:
# Export the City_Data into a csv
complete_df.to_csv("final.csv", encoding="utf-8", index=False)

## 2. Exploratory Data Analysis <a name="EDA"></a>

### 2.1 Descriptive Statistics <a name="DS"></a>

Summary statistics for the dataset were printed.

Knowing the summary statistics for the whole dataset gave a lot of information about each variable and set the basis for a good exploratory analysis.

In [ ]:
complete_df.describe()

Looking at the summary statistics above it was possible to obtain information as the mean, median, standar deviation, min, max, percentiles, of the variables.

In [ ]:
complete_df.describe(include=object)

Summary statistics for object values provided information on the number of unique values in each category and which one is more frequent.
From the table above it could be noticed that the zip code with more sales was 11234, with a frequency of 292.

### 2.2 Correlation <a name="Corr"></a>

Since there were a lot of variables to take into account, correlation helped predict whether some variables had some predictive power over other variables. The correlation coefficient is a statistic that measures the degree to which a variable is a function of some other variable. This value ranges from -1 to 1.

The correlation coefficient between all the variables was calculated to gain a better understanding of which variables were dependent of which other ones.

In [ ]:
def plot_correlation_map(df):
    corr=df.corr()
    _,ax=plt.subplots(figsize=(25,25))
    cmap=sns.diverging_palette(220,10,as_cmap = True )
    _=sns.heatmap(corr,
        cmap=cmap,
        square=True,
        cbar_kws={'shrink':.9},
        ax=ax,
        annot=True,
        annot_kws={'fontsize':15})

plot_correlation_map(complete_df)

### 2.3 Data Visualization <a name="DV"></a>

To gain a better understanding of the correlations, the following code was utilized to create scatter plots between all the variables.
This allowed to better visualize which variables were related and and which relationship could have analyzed more closely.

In [ ]:
sns.pairplot(complete_df, plot_kws={'alpha': 0.3}, diag_kind = 'kde')

A map that shows the Houshold Income in the considered zipcodes was created.

In [ ]:
bins = [1,2,3,4,5,6,7,8,9,10]
complete_df['Household Income Rank'] = pd.cut(complete_df['Household Income'], 
                                              10, labels = bins, ordered = False)
complete_df['Household Income Rank'] = complete_df['Household Income Rank'].astype(int)

zip_map = complete_df[['Zipcode', 'Household Income Rank']]

with urlopen('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/ny_new_york_zip_codes_geo.min.json') as response:
        zipcodes = json.load(response)


fig = px.choropleth(zip_map,
                    geojson=zipcodes,
                    locations='Zipcode',
                    color="Household Income Rank",
                    color_continuous_scale="Viridis",
                    range_color=(1, 10),
                    featureidkey="properties.ZCTA5CE10",
                    scope="usa",
                    labels={"Med Husehold Income": 'Ranked 1 -10'})

fig.update_geos(fitbounds = 'locations')
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

The white spot in the middle represent the zipcode for which no Census Data was available.

In [ ]:
#create new columns for % borrowed, total taxes paid and taxes as a percent of household income and taxes as a percent of gross rent

#divide monetary amounts by 1000 to make the units per 1000
complete_df['% Borrowed'] = complete_df['Amount Mortgaged']/complete_df['Purchase Amt']*100
complete_df['Zip'] = complete_df['Zipcode'].astype(str)
complete_df['Tot tax pd'] = complete_df['Mtg Tx Pd'] + complete_df['Mansion Tax Pd']
complete_df['Mtax as % Household Income'] = complete_df['Mtg Tx Pd']/complete_df['Household Income']* 100
complete_df['Months rent spent on Mtax'] = np.where(complete_df['Mtg Tx Pd'] > 0, complete_df['Median Gross Rent']/complete_df['Mtg Tx Pd'], 0)
complete_df['New Sales Tax Pd % Household Income'] = complete_df['New Sales Tax Amount']/complete_df['Household Income']*100

In [ ]:
#Seperate the data into 2 dataframes, by status
bklyn_investors = complete_df[(complete_df['Owner Occ?'] ==1)]
bklyn_oocc = complete_df[(complete_df['Owner Occ?'] ==0)]

In [ ]:
col_to_plot = ['Purchase Amt', 'Mtg Tx Pd', 'Mtax as % Household Income', 'Months rent spent on Mtax',
       'New Sales Tax Amount','% Borrowed']

def scatter_matrix(col_to_plot, df, output_file):
    fig, axes = plt.subplots(nrows=len(col_to_plot), ncols=2, figsize=(16, 4 * len(col_to_plot)))

    for i, column in enumerate(col_to_plot):
        axes[i, 0].scatter(df['Household Income'], df[column])
        axes[i, 0].set_xlabel('Household Income')
        axes[i, 0].set_ylabel(column)
        correlation = df[column].corr(df['Household Income'])
        axes[i, 0].set_title(f' Household Income vs {column} with a correlation of {correlation:.2f}')

        # Box plot
        axes[i, 1].boxplot(df[column])
        axes[i, 1].set_title(f'Box Plot: {column}')
        axes[i, 1].set_ylabel(column)

    plt.tight_layout()
    plt.savefig(f'{output_file}/scatter_matrix.png')
    plt.show()

scatter_matrix(col_to_plot, bklyn_oocc, 'Images')

This series of scatter plots and box charts shows the relationship between the Median Household Income and Purchase Amt, Mtg Tx Pd, Mtg tax as % Household Income, Months rent spent on Mtg tax, and the New Sales Tax Amount. Pearson's correlation can be found at the top of each chart.

In our pie chart, we showed the revenue generated under the current tax system as a proportion of the new tax regime revenue. To break down the chart, the whole pie shows how much revenue is generated under our proposed tax system. The green slice represents the opportunity cost of staying with the current tax system. The blue and red slice is the tax revenue we currently generate which is only ~66% of what we could generate in our proposed sales tax system.

In [ ]:
mtg_sum = complete_df["Mtg Tx Pd"].sum()
mansion_sum = complete_df["Mansion Tax Pd"].sum()
sales_tax_sum = complete_df["New Sales Tax Amount"].sum()
old_sum = mansion_sum + mtg_sum 
new_sum =  sales_tax_sum 
extra_revenue = new_sum - old_sum

tax_revenue = ["Current Mortgage Tax Paid", "Current Mansion Tax Paid", "Surplus Over Old Tax Collected"]
tax = [mtg_sum, mansion_sum, extra_revenue]
colors = ["lightblue","red","yellowgreen"]

# Tell matplotlib to create a pie chart based upon the above data
plt.pie(tax, labels=tax_revenue, colors=colors,
        autopct="%1.1f%%", shadow=True, startangle=140)
plt.title('Taxes Currently Being Paid As a Portion Proposed New Tax')
# Create axes which are equal so we have a perfect circle
#plt.axis("equal")
# Save an image of our chart and print the final product to the screen
plt.show()

Bar Chart (proportion of taxes paid in the old and new system form Occ vs Investors)

In [ ]:
oocc_mtg_sum = bklyn_oocc["Mtg Tx Pd"].sum()
oocc_mansion_sum = bklyn_oocc["Mansion Tax Pd"].sum()
oocc_sales_tax_sum = bklyn_oocc["New Sales Tax Amount"].sum()
oocc_old_sum = oocc_mansion_sum + oocc_mtg_sum 
investors_mtg_sum = bklyn_investors["Mtg Tx Pd"].sum()
investors_mansion_sum = bklyn_investors["Mansion Tax Pd"].sum()
investors_sales_tax_sum = bklyn_investors["New Sales Tax Amount"].sum()
investors_old_sum = investors_mansion_sum + investors_mtg_sum
bar_labels = ['Old Revenue', 'New Revenue']
y_old_sum = [oocc_old_sum/1e6, investors_old_sum/1e6]
y_new_sales_tax_sum = [oocc_sales_tax_sum/1e6, investors_sales_tax_sum/1e6]

plt.bar(bar_labels, y_old_sum, color='r')
plt.bar(bar_labels, y_new_sales_tax_sum, bottom=y_old_sum, color='g')
plt.title("Tax Revenues")
plt.xlabel("Tax System")
plt.ylabel("Tax Revenues ($1,000,000)")
plt.legend(["Occupied", "Investors"])
plt.show()


Our objective was to demonstrate the comparison of revenue generated under the current tax regime and our proposed system. Our goal was to determine how this system could be advantageous to the state, as well as how it would impact residential home buyers versus investment property owners. The first step was to stratify the Brooklyn data frame into occupied homes and non-occupied homes. Then we were tasked with finding the total tax paid by occupants and investors under the current and new tax systems. We separately calculated the total mortgage tax paid and mansion tax paid by occupants and investors to get the total tax revenue generated. For the new revenue, we calculated the total new sales tax paid by occupants and investors to determine the new revenue generation. After plotting our data as a bar chart, we can see that occupants would pay approximately $10,000,000 less under our proposed tax system, while investors would have to pay approximately $40,000,000 more. This aligns with our objective to tax investment property owners more because today they enjoy many tax benefits, while homeowners bear a heavier burden.

The data was employed to compute the total mortgage tax generated and to project the potential revenue under an alternative scenario involving a sales tax. In this hypothetical scenario, the sales tax was considered at half of the current mortgage tax rate, half of the existing mansion tax rate for primary residence purchases, and half of the prevailing sales tax rate for investment property acquisitions.

In [ ]:
#show the total mortgage tax and mansion taxes collected and compare to what our hypothetical tax would have raised

total_mtg_tax_pd = owner_occ_df['Mtg Tx Pd'].sum()
total_mansion_tax_pd = owner_occ_df['Mansion Tax Pd'].sum()
total_est_tax = owner_occ_df['New Sales Tax Amount'].sum()
total_mtg_tax_inves_pd = investor_df['Mtg Tx Pd'].sum()
total_mansion_tax_inves_pd = investor_df['Mansion Tax Pd'].sum()
total_est_inves_tax = investor_df['New Sales Tax Amount'].sum()
print(f"The total mortgage tax paid in Brooklyn by individuals purchasing primary residences in 2022 was ${total_mtg_tax_pd:,.0f} and the total mansion tax collected was ${total_mansion_tax_pd:,.0f}.")
print(f'The combined mortgage and mansion tax was {total_mtg_tax_pd + total_mansion_tax_pd:,.0f}, our hypothetical sales tax would have raised ${total_est_tax:,.0f}')
print(f"The total mortgage tax paid in Brooklyn by investors in 2022 was ${total_mtg_tax_inves_pd:,.0f} and the total mansion tax collected was ${total_mansion_tax_inves_pd:,.0f}.")
print(f'The combined mortgage and mansion tax was {total_mtg_tax_inves_pd + total_mansion_tax_inves_pd:,.0f}, our hypothetical sales tax on investments would have raised ${total_est_inves_tax:,.0f}')
print(f'The total tax raised in 2022 was {total_mtg_tax_pd + total_mansion_tax_pd + total_mtg_tax_inves_pd + total_mansion_tax_inves_pd:,.0f} and the total amount that would have been raised would have been ${total_est_tax + total_est_inves_tax:,.0f}')

## 3. Conclusions <a name="Concl"></a>

We found upon cleaning our data that our data set was flawed and not as extensive as we initially thought. If we had more time, we believe we might have been able to track down a primary source of data instead of relying on the secondary source that we obtained.
We also, if given more time, would have liked to expand the data to instead be comprised of a random samples of zips codes in all five boroughs and to span different years. The exploration of the data we did procure uncovered information previously unconsidered and lead us to desire further exploration.
Mortgage rates change from year to year and upon examining historical interest rates, we realize that 2022 was far from a typical year. In 2010 the prime rate dropped to 3.25, the lowest it had been in 55 years.  It went up and back down and was again 3.25% in Marck of 2020 where it stayed until March of 2022. Over the course of 2022 the rate continued to climb, reaching 7.5% by the end of 2022. This has a big impact on the types of sales that were completed over the course of that year as the higher interest rates changes individuals ability to qualify for higher purchase amounts.
Another problem with the data is we were not able to obtain a primary data source of the household income for the specific transactions that we studies, we instead relied on the census data for median household income. This data is aggregated by zip and the highest med household income is $173,899 and the biggest purchase amount in the study was $30,563,714. We believe there are many households in Brooklyn with far above $173,899 household income, and no household with that amount of income is purchasing property for over $30 million.

In conclusion, our findings suggest that the new tax system is more equitable because it redistributes the tax burden, alleviating the financial load on residential home buyers while ensuring  that investors contribute a more substantial share. The bar chart analysis involves a comparison of revenue generation between the two systems. Occupants would pay approximately $10,000,000 less under the proposed tax system, while investors would bear an additional cost of around $40,000,000. This stark contrast aligns with the objective of the new tax system to tax investment property owners more, aiming to address the perceived imbalance in tax benefits. In addition, our proposed tax system is more advantageous for the state from a revenue generation standpoint. The opportunity cost of maintaining the current tax system underscores the potential gains and fairness associated with transitioning to the proposed system. This inference is evident through the pie chart we created. The breakdown of revenue in the pie chart indicates that under the current tax system, only approximately 66% of the potential revenue is generated compared to the proposed sales tax system. This highlights an opportunity cost associated with sticking to the existing tax regime.